### Import needed packages and scripts

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pickle, sys, os, json
import astropy.units as u
import pandas as pd
pd.set_option("display.max_columns", None)

from astropy.coordinates import SkyCoord
from matplotlib.dates    import DayLocator, MonthLocator, DateFormatter
from regions             import PointSkyRegion
from astropy.time        import Time
from scipy.stats         import chi2

from gammapy.modeling.models import create_crab_spectral_model, SkyModel, LogParabolaSpectralModel
from gammapy.estimators      import FluxPointsEstimator, LightCurveEstimator, FluxPoints
from gammapy.modeling        import Fit
from gammapy.datasets        import Datasets, SpectrumDataset
from gammapy.makers          import SpectrumDatasetMaker, WobbleRegionsFinder, ReflectedRegionsBackgroundMaker, SafeMaskMaker
from gammapy.maps            import MapAxis, RegionGeom, Map, TimeMapAxis
from gammapy.data            import DataStore

# import scripts
sys.path.insert(0, os.getcwd() + "/../scripts/")
import auxiliar as aux
import plotting

### Paths to data and results

In [2]:
# Root path of this script
root = os.getcwd() + "/"
# Objects directory
root_objects = root + "objects/"
# Data directory
root_data = root + "../../data/"

# Gammapy configuration file
config_gammapy = root_objects + "config_gammapy_analysis.json"

# Path of dl3 data
dl3_dir_paper = "/fefs/aswg/workspace/juan.jimenez/data/tests/crab/dl3_paper/"
#"/fefs/aswg/workspace/daniel.morcuende/data/real/DL3/Crab_performance/AllSkyMC_v0.9.9/intensity80/all_nodes/gh_eff_0.7_th_cont_0.7/"
dl3_dir   = "/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl3/"
#root_data + "cherenkov_transparency_corrections/crab/dl3/"
dl3_dir_s = "/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl3_scaled/"
#root_data + "cherenkov_transparency_corrections/crab/dl3_scaled/"

### Loading configuration files

In [3]:
# Reading the configuration from the gammapy configuration file we created
with open(config_gammapy, "r") as json_file:
    dict = json.load(json_file)

# Saving the configuration in variables
target_name   = dict["target_name"]
n_off_regions = dict["n_off_regions"]
_e_reco = dict["e_reco"]
_e_true = dict["e_true"]

e_reco_min, e_reco_max, e_reco_bin_p_dec = _e_reco["min"], _e_reco["max"], _e_reco["bins_p_dec"]
e_true_min, e_true_max, e_true_bin_p_dec = _e_true["min"], _e_true["max"], _e_true["bins_p_dec"]

### Loading full datasets

In [4]:
# -------------------------------------------------------
# Opening all the dl3 data in a path
total_data_store_paper = DataStore.from_dir(dl3_dir_paper)

# Taking the obs ids
obs_ids_paper = total_data_store_paper.obs_table["OBS_ID"].data
obs_ids_paper = obs_ids_paper[:]

# Then we get the observation information from the total data store
observations_paper = total_data_store_paper.get_observations(
    obs_ids_paper,
    required_irf=["aeff", "edisp", "rad_max"]
)
# -------------------------------------------------------
# -------------------------------------------------------
# Opening all the dl3 data in a path
total_data_store   = DataStore.from_dir(dl3_dir)

# Taking the obs ids
obs_ids = total_data_store.obs_table["OBS_ID"].data
obs_ids = obs_ids[:]

# Then we get the observation information from the total data store
observations = total_data_store.get_observations(
    obs_ids,
    required_irf=["aeff", "edisp", "rad_max"]
)
# -------------------------------------------------------
# -------------------------------------------------------
# Opening all the dl3 data in a path
total_data_store_s = DataStore.from_dir(dl3_dir_s)

# Taking the obs ids
obs_ids_s = total_data_store_s.obs_table["OBS_ID"].data
obs_ids_s = obs_ids_s[:]

# Then we get the observation information from the total data store
observations_s = total_data_store_s.get_observations(
    obs_ids_s,
    required_irf=["aeff", "edisp", "rad_max"]
)
# -------------------------------------------------------

# Defining target position and ON reion
target_position = SkyCoord.from_name(target_name, frame='icrs')
on_region = PointSkyRegion(target_position)

print(f'Total livetime of observations {total_data_store_paper.obs_table["LIVETIME"].data.sum()/3600:.2f} h')
display(total_data_store.obs_table[:5])

Total livetime of observations 18.93 h


OBS_ID,DATE-OBS,TIME-OBS,DATE-END,TIME-END,RA_PNT,DEC_PNT,ZEN_PNT,ALT_PNT,AZ_PNT,RA_OBJ,DEC_OBJ,TSTART,TSTOP,ONTIME,TELAPSE,LIVETIME,DEADC,OBJECT,OBS_MODE,N_TELS,TELLIST,INSTRUME
,,,,,deg,deg,deg,deg,deg,deg,deg,s,s,s,s,s,,,,,,
int64,bytes10,bytes12,bytes10,bytes12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,bytes8,int64,bytes5,bytes5
2914,2020-11-18,03:43:34.194,2020-11-18,04:02:52.571,83.28333315577058,21.78656761995211,14.827836028194994,75.172163971805,245.10365521538614,83.6287,22.0147,67319014.1938901,67320172.57093215,1158.378259897232,1158.37704205513,1110.7061580196348,0.9588458247810809,crab,POINTING,1,LST-1,LST-1
2929,2020-11-19,01:41:13.090,2020-11-19,01:59:21.736,83.97361218928079,22.244784527308934,15.852691452043658,74.14730854795634,110.69510910121441,83.6287,22.0147,67398073.0903821,67399161.73551226,1088.6460654735565,1088.6451301574707,1006.0309340133695,0.9241120378051889,crab,POINTING,1,LST-1,LST-1
2930,2020-11-19,02:02:55.113,2020-11-19,02:22:43.860,83.26877581309876,21.786545814732005,11.237297662536207,78.7627023374638,126.56517001542876,83.6287,22.0147,67399375.11288929,67400563.8598373,1187.369912147522,1188.746948003769,1106.9165247250533,0.9322423563209904,crab,POINTING,1,LST-1,LST-1
2931,2020-11-19,02:24:41.947,2020-11-19,02:44:15.865,83.9756101018619,22.244460939543412,7.96866394897576,82.03133605102424,145.00062108158266,83.6287,22.0147,67400681.94708037,67401855.86515117,1172.8647966384888,1173.9180707931519,1097.3626797944742,0.9356258990291048,crab,POINTING,1,LST-1,LST-1
2932,2020-11-19,02:46:09.185,2020-11-19,03:05:51.191,83.27280963639575,21.776677987260904,7.155731807508118,82.84426819249188,188.473005543687,83.6287,22.0147,67401969.18531418,67403151.19075465,1172.2196559906006,1182.0054404735565,1094.4444604410246,0.9336513466976025,crab,POINTING,1,LST-1,LST-1


### Defining all the energy axes

In [5]:
# ============================ #
# estimated energy axes
energy_axis = MapAxis.from_energy_bounds(
    e_reco_min, e_reco_max, 
    nbin=e_reco_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy"
)
# ============================ #
# estimated energy axes
energy_axis_true = MapAxis.from_energy_bounds(
    e_true_min, e_true_max, 
    nbin=e_true_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy_true"
)
# ============================ #
# Energy for the spectrum
e_fit_min = energy_axis.edges[0].value
e_fit_max = energy_axis.edges[-1].value
e_fit_bin_p_dec = e_reco_bin_p_dec

# Just to have a separate MapAxis for spectral fit energy range
energy_fit_edges = MapAxis.from_energy_bounds(
    e_fit_min, e_fit_max, 
    nbin=e_fit_bin_p_dec, per_decade=True, 
    unit="TeV"
).edges

# ============================ #
# Energy for the lightcurve
e_lc_min = energy_axis.edges[0]
e_lc_max = energy_axis.edges[-1]

print("Spectral fit will be done in energy edges:\n", energy_fit_edges)
print(f"\nLC will be estimated from {e_lc_min:.1f} to {e_lc_max:.1f}")

Spectral fit will be done in energy edges:
 [ 0.25        0.38876979  0.60456779  0.94015077  1.46200887  2.27353952
  3.53553391  5.49803508  8.54987973 13.29573974 20.67592771 32.15270417
 50.        ] TeV

LC will be estimated from 0.2 TeV to 50.0 TeV


### We define the geometry regions in te sky and prepare the empty datasets and makers

In [6]:
# geometry defining the ON region and SpectrumDataset based on it
geom = RegionGeom.create(
    region=on_region, 
    axes=[energy_axis]
)

# -------------------------------------------------------
# creating an empty dataset
dataset_empty_paper = SpectrumDataset.create(
    geom=geom, 
    energy_axis_true=energy_axis_true
)
dataset_maker_paper = SpectrumDatasetMaker(
    containment_correction=False,
    selection=["counts", "exposure", "edisp"]
)
# -------------------------------------------------------
# -------------------------------------------------------
# creating an empty dataset
dataset_empty = SpectrumDataset.create(
    geom=geom, 
    energy_axis_true=energy_axis_true
)
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False,
    selection=["counts", "exposure", "edisp"]
)
# -------------------------------------------------------
# -------------------------------------------------------
dataset_empty_s = SpectrumDataset.create(
    geom=geom, 
    energy_axis_true=energy_axis_true
)
dataset_maker_s = SpectrumDatasetMaker(
    containment_correction=False,
    selection=["counts", "exposure", "edisp"]
)
# -------------------------------------------------------

# tell the background maker to use the WobbleRegionsFinder
region_finder = WobbleRegionsFinder(n_off_regions=n_off_regions)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

### Now we analize the ON and OFF regions in the dataset and we store them in `datasets`, then the datasets can be stacked in a unique one

In [ ]:
%%time
# -------------------------------------------------------
# The final object will be stored as a Datasets object
datasets_paper = Datasets()
for observation_paper in observations_paper:
    dataset_paper = dataset_maker_paper.run(
        dataset=dataset_empty_paper.copy(name=str(observation_paper.obs_id)),
        observation=observation_paper
    )
    dataset_on_off_paper = bkg_maker.run(
        dataset=dataset_paper, 
        observation=observation_paper
    )
    datasets_paper.append(dataset_on_off_paper) 

# Stacking all the datasets in one
stacked_dataset_paper = Datasets(datasets_paper).stack_reduce()
print(stacked_dataset_paper)
# -------------------------------------------------------
# -------------------------------------------------------
# The final object will be stored as a Datasets object
datasets = Datasets()
for observation in observations:
    dataset = dataset_maker.run(
        dataset=dataset_empty.copy(name=str(observation.obs_id)),
        observation=observation
    )
    dataset_on_off = bkg_maker.run(
        dataset=dataset, 
        observation=observation
    )
    datasets.append(dataset_on_off) 

# Stacking all the datasets in one
stacked_dataset = Datasets(datasets).stack_reduce()
print(stacked_dataset)
# -------------------------------------------------------
# -------------------------------------------------------
# The final object will be stored as a Datasets object
datasets_s = Datasets()
for observation_s in observations_s:
    dataset_s=dataset_maker_s.run(
        dataset=dataset_empty_s.copy(name=str(observation_s.obs_id)),
        observation=observation_s
    )
    dataset_on_off_s = bkg_maker.run(
        dataset=dataset_s, 
        observation=observation_s
    )
    datasets_s.append(dataset_on_off_s)

# Stacking all the datasets in one
stacked_dataset_s = Datasets(datasets_s).stack_reduce()
print(stacked_dataset_s)
# -------------------------------------------------------

SpectrumDatasetOnOff
--------------------

  Name                            : Y8ulGUCx 

  Total counts                    : 23030 
  Total background counts         : 5318.00
  Total excess counts             : 17712.00

  Predicted counts                : 14174.00
  Predicted background counts     : 14174.00
  Predicted excess counts         : nan

  Exposure min                    : 2.47e+07 m2 s
  Exposure max                    : 1.36e+10 m2 s

  Number of total bins            : 12 
  Number of fit bins              : 12 

  Fit statistic type              : wstat
  Fit statistic value (-2 log(L)) : 12884.73

  Number of models                : 0 
  Number of parameters            : 0
  Number of free parameters       : 0

  Total counts_off                : 5318 
  Acceptance                      : 12 
  Acceptance off                  : 12 



### Then we define the model and set inside the dataset

In [ ]:
# defining the model we want to fit and the starting values
spectral_model = LogParabolaSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    alpha=2,
    beta=0.1,
    reference=1 * u.TeV,
)
# -------------------------------------------------------
# we will use the crab model in general
model_paper = SkyModel(
    spectral_model=spectral_model, 
    name="crab"
)
# We set the model of all datasets to log parabola
stacked_dataset_paper.models = model_paper
# -------------------------------------------------------
# -------------------------------------------------------
# we will use the crab model in general
model = SkyModel(
    spectral_model=spectral_model, 
    name="crab"
)
# We set the model of all datasets to log parabola
stacked_dataset.models = model
# -------------------------------------------------------
# -------------------------------------------------------
# we will use the crab model in general
model_s = SkyModel(
    spectral_model=spectral_model, 
    name="crab"
)
# We set the model of all datasets to log parabola
stacked_dataset_s.models = model_s
# -------------------------------------------------------

### We fit the model with the stacked dataset

In [ ]:
%%time
# -------------------------------------------------------
# Now we run the fit to extract the parameters of the model
fit_paper = Fit()
result_paper = fit_paper.run(datasets=stacked_dataset_paper)
best_fit_model_paper = model_paper.copy()
display(stacked_dataset_paper.models.to_parameters_table())
# -------------------------------------------------------
# -------------------------------------------------------
# Now we run the fit to extract the parameters of the model
fit = Fit()
result = fit.run(datasets=stacked_dataset)
best_fit_model = model.copy()
display(stacked_dataset.models.to_parameters_table())
# -------------------------------------------------------
# -------------------------------------------------------
# Now we run the fit to extract the parameters of the model
fit_s = Fit()
result_s = fit_s.run(datasets=stacked_dataset_s)
best_fit_model_s = model_s.copy()
display(stacked_dataset_s.models.to_parameters_table())
# -------------------------------------------------------

### Then from the model and the data we can extract the flux points

In [ ]:
%%time
# then extracting the flux points from the data
fpe = FluxPointsEstimator(
    energy_edges=energy_fit_edges, 
    source=target_name, 
    selection_optional="all"
)

# -------------------------------------------------------
# We apply the flux point estiation from the datasets
flux_points_paper = fpe.run(datasets=stacked_dataset_paper)
display(flux_points_paper.to_table(sed_type="dnde", formatted=True)[:3])
# -------------------------------------------------------
# -------------------------------------------------------
# We apply the flux point estiation from the datasets
flux_points = fpe.run(datasets=stacked_dataset)
display(flux_points.to_table(sed_type="dnde", formatted=True)[:3])
# -------------------------------------------------------
# -------------------------------------------------------
# We apply the flux point estiation from the datasets
flux_points_s = fpe.run(datasets=stacked_dataset_s)
display(flux_points_s.to_table(sed_type="dnde", formatted=True)[:3])
# -------------------------------------------------------

### Then we can plot the SED

In [ ]:
plot_kwargs = {
    "energy_bounds": [0.08, 100] * u.TeV,
    "sed_type": "e2dnde",
    "yunits": u.Unit("TeV cm-2 s-1"),
    "xunits": u.TeV,
}
cmap = plotting.create_cmap_from_colors(["w", "w", "cornflowerblue"])

# Crab models
crab_magic_100 = create_crab_spectral_model("magic_lp")



fig, ax = plt.subplots(figsize=(10,5))

best_fit_model_paper.spectral_model.plot_error(
    ax=ax, facecolor="k", alpha=0.2, **plot_kwargs,
)
best_fit_model.spectral_model.plot_error(
    ax=ax, facecolor="r", alpha=0.2, **plot_kwargs
)
best_fit_model_s.spectral_model.plot_error(
    ax=ax, facecolor="b", alpha=0.2, **plot_kwargs
)
best_fit_model_s.spectral_model.plot_error(
    ax=ax, facecolor="mediumblue", alpha=0.2, **plot_kwargs
)

crab_magic_100.plot(
    ax=ax, ls="--", lw=1.5, color="crimson", label="MAGIC reference (Aleksić et al. 2015)", **plot_kwargs
)

flux_points_paper.plot(sed_type="e2dnde", color="k", label="Flux points performance paper")
flux_points.plot(sed_type="e2dnde", color="r", label="Flux points original data")
flux_points_s.plot(sed_type="e2dnde", color="b", marker="o", label="Flux points scaled data")
# flux_points_s.plot_ts_profiles(sed_type="e2dnde", cmap=cmap)

ax.legend(loc=3, frameon=False)
ax.set_ylim([1e-13, 1e-10])
ax.grid(which="both", alpha=0.5)
ax.set_ylabel("$E^2\\frac{dN}{dE}$ [TeVs${}^{-1}$cm${}^{-2}$]")

plt.savefig(f"plots/total.png", bbox_inches="tight", dpi=400)
plt.show()

### Then once we have found the SED model we fix the alpha and beta parameters and let the amplitude as a free parameter

In [ ]:
# Create the LC Estimator for each run
lc_maker_1d = LightCurveEstimator(
    energy_edges=[e_lc_min, e_lc_max], 
    reoptimize=False, # Re-optimizing other free model parameters (not belonging to the source)
    source="crab", 
    selection_optional="all" # Estimates asymmetric errors, upper limits and fit statistic profiles
)

# -------------------------------------------------------
model_paper.parameters["alpha"].frozen = True
model_paper.parameters["beta"].frozen  = True
# Assigning the fixed parameters model to each dataset
for data in datasets_paper:
    data.models = model_paper
# -------------------------------------------------------
# -------------------------------------------------------
model.parameters["alpha"].frozen = True
model.parameters["beta"].frozen  = True
# Assigning the fixed parameters model to each dataset
for data in datasets:
    data.models = model
# -------------------------------------------------------
# -------------------------------------------------------
model_s.parameters["alpha"].frozen = True
model_s.parameters["beta"].frozen  = True
# Assigning the fixed parameters model to each dataset
for data in datasets_s:
    data.models = model_s
# -------------------------------------------------------

### Then we run the lightcurve maker run-wise

In [ ]:
%%time
print(f"LC will be estimated from {e_lc_min:.1f} to {e_lc_max:.1f}")

# -------------------------------------------------------
lc_runwise_paper = lc_maker_1d.run(datasets_paper)
lightcurve_paper = lc_runwise_paper.to_table(sed_type="flux", format="lightcurve")
# -------------------------------------------------------
# -------------------------------------------------------
lc_runwise = lc_maker_1d.run(datasets)
lightcurve = lc_runwise.to_table(sed_type="flux", format="lightcurve")
# -------------------------------------------------------
# -------------------------------------------------------
lc_runwise_s = lc_maker_1d.run(datasets_s)
lightcurve_s = lc_runwise_s.to_table(sed_type="flux", format="lightcurve")
# -------------------------------------------------------

### We calculate the mean flux and the statistical error
### Calculating the means, errors, $\chi^2$ and $p$ values

In [ ]:
def weighted_average(table, sys_error=0):
    val = table["flux"]
    uncertainty = np.sqrt((sys_error * table["flux"])**2 + table["flux_err"]**2)
    return (val/uncertainty**2).sum() / (1/uncertainty**2).sum(), np.sqrt(1/np.sum(1/uncertainty**2))

def calculate_chi2_pvalue(table, sys_error=0):
    uncertainty = np.sqrt((sys_error * table["flux"])**2 + table["flux_err"]**2)
    flux = table["flux"]
    mean_flux = (flux/uncertainty**2).sum() / (1/uncertainty**2).sum()
    mean_flux_err = np.sqrt(1/np.sum(1/uncertainty**2))
    print(f"Weighted mean flux: {mean_flux:.3e} +/- {mean_flux_err:.3e} cm-2 s-1")
    
    chi2_value = np.sum((table["flux"] - mean_flux)**2/uncertainty**2)
    ndf = len(table["flux"]) - 1
    pvalue = chi2.sf(x=chi2_value, df=ndf)
    print(f"Chi2: {chi2_value:.1f}, ndf: {ndf}, P-value: {pvalue:.2e}")
    return chi2_value, ndf, pvalue

# -------------------------------------------------------
mean_flux_paper, mean_flux_err_paper    = weighted_average(lightcurve_paper)
chi2_val_paper, ndf_paper, pvalue_paper = calculate_chi2_pvalue(lightcurve_paper, sys_error=0.0)
# -------------------------------------------------------
# -------------------------------------------------------
mean_flux,   mean_flux_err   = weighted_average(lightcurve)
chi2_val,   ndf,   pvalue   = calculate_chi2_pvalue(lightcurve, sys_error=0.0)
# -------------------------------------------------------
# -------------------------------------------------------
mean_flux_s, mean_flux_err_s = weighted_average(lightcurve_s)
chi2_val_s, ndf_s, pvalue_s  = calculate_chi2_pvalue(lightcurve_s, sys_error=0.0)
# -------------------------------------------------------

### Extracting the data from the table as arrays

In [ ]:
# -------------------------------------------------------
time_min_paper = Time(np.hstack(lightcurve_paper["time_min"]), format='mjd').datetime
time_max_paper = Time(np.hstack(lightcurve_paper["time_max"]), format='mjd').datetime
delta_time_paper  = time_max_paper - time_min_paper
time_center_paper = time_min_paper + delta_time_paper / 2
# Flux and flux error
flux_lst1_paper = np.hstack(lightcurve_paper["flux"])
flux_stat_err_lst1_paper = np.hstack(lightcurve_paper["flux_err"])
# run numbers
run_num_paper = [int(n) for n in observations_paper.ids]
# -------------------------------------------------------
# -------------------------------------------------------
time_min = Time(np.hstack(lightcurve["time_min"]), format='mjd').datetime
time_max = Time(np.hstack(lightcurve["time_max"]), format='mjd').datetime
delta_time  = time_max - time_min
time_center = time_min + delta_time / 2
# Flux and flux error
flux_lst1 = np.hstack(lightcurve["flux"])
flux_stat_err_lst1 = np.hstack(lightcurve["flux_err"])
# run numbers
run_num = [int(n) for n in observations.ids]
# -------------------------------------------------------
# -------------------------------------------------------
time_min_s = Time(np.hstack(lightcurve_s["time_min"]), format='mjd').datetime
time_max_s = Time(np.hstack(lightcurve_s["time_max"]), format='mjd').datetime
delta_time_s  = time_max_s - time_min_s
time_center_s = time_min_s + delta_time_s / 2
# Flux and flux error
flux_lst1_s = np.hstack(lightcurve_s["flux"])
flux_stat_err_lst1_s = np.hstack(lightcurve_s["flux_err"])
# run numbers
run_num_s = [int(n) for n in observations_s.ids]
# -------------------------------------------------------

### The Crab Nebula reference from MAGIC

In [ ]:
crab = create_crab_spectral_model("magic_lp")

crab.amplitude.error = 0.03e-11 * u.Unit("cm-2 s-1 TeV-1")
crab.alpha.error = 0.01
crab.beta.error = 0.01/np.log(10)

flux_crab = crab.integral(e_lc_min, e_lc_max)
flux_crab_error = flux_crab * 0

### Plotting the LC

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

# Plotting the Ligt Curve
ax.errorbar(time_center_paper, flux_lst1_paper, yerr=flux_stat_err_lst1_paper, color="k", ls="", marker=".", ms=10, label="LST-1 (src-independent) original")
ax.errorbar(time_center, flux_lst1, yerr=flux_stat_err_lst1, color="r", ls="", marker="x", ms=10, label="LST-1 (src-independent) scaled")
ax.errorbar(time_center_s, flux_lst1_s, yerr=flux_stat_err_lst1_s, color="b", ls="", marker=".", ms=10, label="LST-1 (src-independent) scaled")
    
# Mean flux + error
ax.axhline(mean_flux_paper, ls="--", color="k", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux_paper - mean_flux_err_paper, mean_flux_paper + mean_flux_err_paper, color="k", alpha=0.4, zorder=-1)
ax.axhline(mean_flux, ls="--", color="r", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux - mean_flux_err, mean_flux + mean_flux_err, color="r", alpha=0.4, zorder=-1)
ax.axhline(mean_flux_s, ls="--", color="b", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux_s - mean_flux_err_s, mean_flux_s + mean_flux_err_s, color="b", alpha=0.4, zorder=-1)

# MAGIC reference
ax.axhline(flux_crab.value, ls="-.", color="crimson", zorder=-1, label="MAGIC (Aleksić et al. 2015)")

energy_range = f"{e_lc_min:.2f} < $E$ < {e_lc_max:.0f}"
ax.set_title(f"Light curve of {target_name} ({energy_range})")
ax.set_ylabel("Flux [cm$^{-2}$ s$^{-1}$]")
ax.set_xlabel("Time [date]")
# ax.grid()
ax.legend(loc=4)
# ax.set_xlim(18876, 18882.5)
ax.xaxis.set_major_locator(MonthLocator(interval=2))
ax.xaxis.set_major_formatter(DateFormatter("%d-%m-%Y"))
plt.xticks(rotation=45)
ax.set_ylim(0)

# plt.savefig(f"plots/total.png", bbox_inches="tight", dpi=300)
plt.show()

### Plotting the LC run-wise

In [ ]:
fig, ax = plt.subplots(figsize=(17,3))

# Plotting the Ligt Curve
ax.errorbar(np.arange(len(flux_lst1_paper)), flux_lst1_paper, yerr=flux_stat_err_lst1_paper, color="k", ls="", marker=".", ms=10, label="LST-1 (src-independent)")  
ax.errorbar(np.arange(len(flux_lst1)), flux_lst1, yerr=flux_stat_err_lst1, color="r", ls="", marker=".", ms=10, label="LST-1 (src-independent)")  
ax.errorbar(np.arange(len(flux_lst1_s)), flux_lst1_s, yerr=flux_stat_err_lst1_s, color="b", ls="", marker=".", ms=10, label="LST-1 (src-independent)")  

# Mean flux + error
ax.axhline(mean_flux_paper, ls="--", color="k", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux_paper - mean_flux_err_paper, mean_flux_paper + mean_flux_err_paper, color="k", alpha=0.4, zorder=-1)
ax.axhline(mean_flux, ls="--", color="r", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux - mean_flux_err, mean_flux + mean_flux_err, color="r", alpha=0.4, zorder=-1)
ax.axhline(mean_flux_s, ls="--", color="b", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux_s - mean_flux_err_s, mean_flux_s + mean_flux_err_s, color="b", alpha=0.4, zorder=-1)
# MAGIC reference
ax.axhline(flux_crab.value, ls="-.", color="crimson", zorder=-1, label="MAGIC (Aleksić et al. 2015)")

# ax.legend()
ax.grid()
ax.set_xlabel(f"Run #")
ax.set_ylabel("Flux [cm$^{-2}$ s$^{-1}$]")
energy_range = f"{e_lc_min:.1f} < $E$ < {e_lc_max:.1f}"
ax.set_title(f"Light curve of {target_name} ({energy_range})")
# plt.savefig(f"plots/total.png", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
np.std(flux_lst1_paper), np.std(flux_lst1), np.std(flux_lst1_s)

### Now we put all inside a dict and we store it in a file

In [ ]:
dict_total = {
    
    "dict_model" : best_fit_model.to_dict(), # SkyModel.from_dict(<>)
    
    "table_sed"  : flux_points.to_table(),   # FluxPoints.from_table(<>)

    "lightcurve" : {

        "run_number" : run_num,
        "t_start"    : time_min,
        "t_stop"     : time_max,
        "timedelta"  : delta_time,
        "flux"       : flux_lst1,
        "e_flux"     : flux_stat_err_lst1,
        
        "global" : {
            "e_min"               : e_lc_min,
            "e_max"               : e_lc_max,
            "n_off_regions"       : n_off_regions,
            "target_name"         : target_name,
            "crab_reference_flux" : flux_crab,
            "chi2"                : chi2_val,
            "pvalue"              : pvalue
        }
    }
}


# Saving the object
# with open(fname_dict, 'wb') as f:
#     pickle.dump(dict_total, f, pickle.HIGHEST_PROTOCOL)